<a href="https://colab.research.google.com/github/bbohrer6/TransferLearning_CATDog.ipynb/blob/main/TransferLearning_CATDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/treino',  # Substitua pelo caminho da sua pasta 'treino' no Drive
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/teste',  # Substitua pelo caminho da sua pasta 'teste' no Drive
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 77 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


In [11]:
# Carregar a rede pré-treinada VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar as camadas da base
for layer in base_model.layers:
    layer.trainable = False

# Adicionar camadas densas para classificação
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x) # Use 'softmax' para mais de duas classes

# Criar o modelo final
model = Model(inputs=base_model.input, outputs=x)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10, # Ajuste o número de épocas
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 123s 107s/step - accuracy: 0.5350 - loss: 4.3047 - val_accuracy: 0.5312 - val_loss: 2.3384
Epoch 2/10
1/2 ━━━━━━━━━━━━━━━━━━━━ 19s 20s/step - accuracy: 0.6562 - loss: 1.4943

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2/2 ━━━━━━━━━━━━━━━━━━━━ 42s 22s/step - accuracy: 0.6562 - loss: 1.4943 - val_accuracy: 0.5625 - val_loss: 3.3167
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 142s 73s/step - accuracy: 0.4169 - loss: 4.0592 - val_accuracy: 0.7500 - val_loss: 0.4301
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 82s 61s/step - accuracy: 0.8750 - loss: 0.3106 - val_accuracy: 0.5312 - val_loss: 2.2170
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 144s 95s/step - accuracy: 0.6250 - loss: 1.5539 - val_accuracy: 0.3750 - val_loss: 2.1598
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 39s 32s/step - accuracy: 0.5385 - loss: 1.4852 - val_accuracy: 0.8125 - val_loss: 0.2994
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 82s 67s/step - accuracy: 0.7966 - loss: 0.5396 - val_accuracy: 0.5625 - val_loss: 1.1528
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 44s 22s/step - accuracy: 0.6562 - loss: 0.9045 - val_accuracy: 0.6875 - val_loss: 0.6859
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 63s 35s/step - accuracy: 0.8694 - loss: 0.3938 - val_accuracy: 0.9062 - val_loss: 0.2130
Epoch 10/

In [14]:
loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print('Loss:', loss)
print('Accuracy:', accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 25s 25s/step - accuracy: 0.7812 - loss: 0.5823
Loss: 0.5823318362236023
Accuracy: 0.78125


In [27]:
img_path = '/content/drive/MyDrive/Dataset/shih-tzu.webp'  # Substitua pelo caminho da sua imagem no Drive

In [28]:
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
import numpy as np

# ... (código para definir img_path) ...

# Carrega a imagem
img = load_img(img_path, target_size=(224, 224))

# Converte a imagem para um array NumPy usando img_to_array
img_array = img_to_array(img)  # Correção: chama a função e armazena o resultado

# Pré-processa a imagem
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

# ... (código para fazer a previsão) ...

In [29]:
prediction = model.predict(img_array)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 571ms/step
[[2.7879445e-08]]


In [39]:
!git clone https://github.com/bbohrer6/TransferLearning_CATDog.git

Cloning into 'TransferLearning_CATDog'...
fatal: could not read Username for 'https://github.com': No such device or address
